In [1]:
%matplotlib inline

import os,sys

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
from torch.autograd import Variable

import timeit

%load_ext autoreload
%autoreload 2

In [2]:
def apply_mirror_boundary_conditions(coord, dim):
    """
    Return the correct coordinate according to mirror boundary conditions
        coord: a coordinate (x or y) in the image
        dim: the length of the axis of said coordinate
    """
    # If the coordinate is outside of the bounds of the axis, take its reflection inside the image
    if coord < 0:
        coord = -coord
    elif coord >= dim:
        coord =  2*(dim-1) - coord % (2*(dim-1))
    # Else, do nothing
    return int(coord)

def get_window(image, window_size, centre_coordinates):
    """
    Get a window in image taking into account boundary conditions
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
        centre_coordinates: a list containing the x-y coordinates of the window's central pixel
    """
    # Get convenient variables
    window_radius = (window_size - 1)/2
    i_centre, j_centre = (centre_coordinates[0], centre_coordinates[1])
    nrows, ncols, nchannels = image.shape
    window = np.zeros((window_size, window_size, nchannels))
    # Fill in the window array with pixels of the image
    for i in range(window_size):
        # Apply mirror boundary conditions on the x-coordinate
        i_mirrored = apply_mirror_boundary_conditions(i_centre + i - window_radius, nrows)
        for j in range(window_size):
            # Same for the y-coordinate
            j_mirrored = apply_mirror_boundary_conditions(j_centre + j - window_radius, ncols)
            # Fill in the window with the corresponding pixel
            window[i, j, :] = image[i_mirrored, j_mirrored, :]
    return window

In [53]:
for j in range(4, 3, -1):
    print(j)

4


In [65]:
def shift_to_the_right(image, window, centre_coordinates, step=1):
    nrows, ncols, _ = image.shape
    window_size = len(window)
    window_radius = (window_size - 1)/2
    #j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + step + window_radius, ncols)
    shifted = np.roll(window, -step, axis=1)
    for i in range(window_size):
        i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + i - window_radius, nrows)            
        for j in range(window_size-step, window_size):
            j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + j - window_radius + step, ncols)
            shifted[i, j, :] = image[i_mirrored, j_mirrored, :]
    return shifted

def shift_to_the_bottom(image, window, centre_coordinates, step=1):
    nrows, ncols, _ = image.shape
    window_size = len(window)
    window_radius = (window_size - 1)/2
    #i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + 1 + window_radius, nrows)
    shifted = np.roll(window, -step, axis=0)
    #print('shifted\n', shifted[:,:,0], '\n')
    for j in range(window_size):
        j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + j - window_radius, ncols)
        for i in range(window_size-step, window_size):
            i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + i - window_radius + step, nrows)
            #print(i, i_mirrored)
            shifted[i, j, :] = image[i_mirrored, j_mirrored, :]
    return shifted

def sliding_window(image, window_size, step=1):
    """
    Construct a list of sliding windows of given size on an image.
    The windows will slide from left to right and from up to down.
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
        step: the value of the shift between windows
    """
    nrows, ncols, _ = image.shape
    windows = []
    i = 0
    row_windows = [get_window(image, window_size, [i, 0])]
    for j in range(0, ncols-1, step):
        row_windows += [shift_to_the_right(image, row_windows[-1], [i, j], step)]
    windows += row_windows
    for i in range(0, nrows-1, step):
        row_windows = [shift_to_the_bottom(image, row_windows[int(j/step)], [i, j], step) for j in range(0, ncols, step)]
        windows += row_windows
    return windows

In [67]:
# Directory and files name
train_dir = "training/"
tr_image_dir = train_dir + "images/"
tr_label_dir = train_dir + "groundtruth/"

tr_image_files = os.listdir(tr_image_dir)
tr_label_files = os.listdir(tr_label_dir)

# Number of training samples
N = len(tr_image_files)

# Load the images and ground truth
img_train = []
label_train = []
for i in range(N):
    img = mpimg.imread(tr_image_dir + tr_image_files[i])
    label = mpimg.imread(tr_label_dir + tr_label_files[i])
    
    img_train.append(img)
    label_train.append(label)

# Keep only sub-set of images
NUM_IMAGES = N

img_train = np.asarray(img_train[:NUM_IMAGES])
label_train = np.asarray(label_train[:NUM_IMAGES])

print(img_train.shape, label_train.shape)

(100, 400, 400, 3) (100, 400, 400)


In [31]:
im = img_train[0]
for window_size in [3, 5, 7, 15, 35, 51]:
    %timeit sliding_window(im, window_size, step=int(window_size/2))

19.8 s ± 1.89 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.87 s ± 397 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.97 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1 s ± 77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
367 ms ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
220 ms ± 43.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
def compute_all_windows(img_train, label_train, window_size, step):
    for im, labels, i in zip(img_train, label_train, range(len(img_train))):
        w_im = sliding_window(im, window_size, step)
        w_labels = sliding_window(labels[:, :, np.newaxis], window_size, step)
        path = './windows_train/' + str(i)
        os.makedirs(path, exist_ok=True)
        for wi, wl, j in zip(w_im, w_labels, range(len(w_im))):
            img_name = path + '/im_' + str(j) + '.png'
            plt.imsave(img_name, wi)
            label_name = path + '/label_' + str(j) + '.png'
            plt.imsave(label_name, wl[:,:,0])

In [68]:
window_size = 51
step = int(window_size/2)
compute_all_windows(img_train, label_train, window_size, step)

In [64]:
a = np.array(range(10*10))
a = a.reshape((10, 10))
print(a)
aa = get_window(a[:,:,np.newaxis], 5, [5,5])[:,:,0]
print('\n')
print(aa)
print('\n')
print(shift_to_the_bottom(a[:,:,np.newaxis], aa[:,:,np.newaxis], [5,5], 2)[:,:,0])

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


[[ 33.  34.  35.  36.  37.]
 [ 43.  44.  45.  46.  47.]
 [ 53.  54.  55.  56.  57.]
 [ 63.  64.  65.  66.  67.]
 [ 73.  74.  75.  76.  77.]]


shifted
 [[ 53.  54.  55.  56.  57.]
 [ 63.  64.  65.  66.  67.]
 [ 73.  74.  75.  76.  77.]
 [ 33.  34.  35.  36.  37.]
 [ 43.  44.  45.  46.  47.]] 

3 8
4 9
3 8
4 9
3 8
4 9
3 8
4 9
3 8
4 9
[[ 53.  54.  55.  56.  57.]
 [ 63.  64.  65.  66.  67.]
 [ 73.  74.  75.  76.  77.]
 [ 83.  84.  85.  86.  87.]
 [ 93.  94.  95.  96.  97.]]
